In [258]:
import numpy as np
import pandas as pd

In [259]:
# 파일 불러오기
file_path = "C:/Users/82108/Desktop/jw/프로젝트/반려동물등록현황전처리전/서울_동물운송업_영업장/서울_동물운송업_영업장목록_20240427 - 수정본.csv"
df_raw = pd.read_csv(file_path)
df_raw 

,번호,구분,업체명,소재지,인허가 정보
0,409,운송,pet131나1455,서울특별시 강동구 길동 382번지 6호 우성아파트,2024
1,410,운송,pet04모2461,서울특별시 강동구 명일동 352번지 3호 명일노블리안,2024
2,411,운송,폰토스 인터네셔널,서울특별시 강동구 성내동 562번지 10호 삼성아트빌,2024
3,412,운송,pet66도 8410,서울특별시 강동구 강일동 91번지 강동리버스트7단지 702 736,2024
4,413,운송,pet116주7521,서울특별시 강동구 성내동 405번지 5호 한솔애리즈아파트,2023
...,...,...,...,...,...
431,10,운송,이예슬펫택시,서울특별시 용산구 한강로2가 2번지 185호 용산아이피아대주피오레,2020
432,11,운송,타다펫,서울특별시 용산구 보광동 78번지 4호,2018
433,3,운송,엔젤,서울특별시 중구 신당동 844번지 29통 2반,2019
434,1,운송,486펫콜 서울종로점,서울특별시 종로구 충신동 25번지 73호,2022


In [260]:
# 원본데이터 복사
transportation = df_raw.copy()

In [261]:
# 필요한 열만 남기기
transportation = transportation[['구분','업체명','소재지','인허가 정보']]
transportation.head(3)

,구분,업체명,소재지,인허가 정보
0,운송,pet131나1455,서울특별시 강동구 길동 382번지 6호 우성아파트,2024
1,운송,pet04모2461,서울특별시 강동구 명일동 352번지 3호 명일노블리안,2024
2,운송,폰토스 인터네셔널,서울특별시 강동구 성내동 562번지 10호 삼성아트빌,2024


In [262]:
# 각 행정구와 동만 추출하기
transportation['행정구'] = transportation['소재지'].str.split(' ').str[1]
transportation['동'] = transportation['소재지'].str.split(' ').str[2]

# 소재지 열 제거하기
transportation = transportation[['행정구','동','구분','인허가 정보']]

# 열 이름 바꾸기
transportation.rename(columns={'행정구':'Gu',
                        '동':'Dong',
                        '구분':'Category',
                        '인허가 정보':'Current_License_info'},
                  inplace=True)

# 데이터 형태 보기
transportation.head()

,Gu,Dong,Category,Current_License_info
0,강동구,길동,운송,2024
1,강동구,명일동,운송,2024
2,강동구,성내동,운송,2024
3,강동구,강일동,운송,2024
4,강동구,성내동,운송,2023


In [263]:
# 구, 동, 카테고리, 인허가 정보 별 수
# df: 구, 동, 카테고리, 인허가 정보를 그룹화한 수를 포함
df = transportation.groupby(['Gu', 'Dong', 'Category', 'Current_License_info']).size()
df

Gu   Dong  Category  Current_License_info
강남구  논현동   운송        2018                    2
                     2019                    1
                     2021                    2
                     2022                    1
                     2024                    1
                                            ..
중랑구  면목동   운송        2024                    1
     묵동    운송        2022                    1
                     2023                    2
     신내동   운송        2022                    1
     중화동   운송        2023                    1
Length: 298, dtype: int64

In [264]:
# 데이터를 데이터 프레임 형식으로 변경
df = pd.DataFrame(df)

# 기존 인덱스를 열로 이동하고, 새로운 정수 인덱스 할당
df = df.reset_index()

# groupby로 구해진 수의 열을 Count라는 이름으로
# 인덱스에 idx라는 이름 할당
# rename_axis()메서드를 사용하여 인덱스 이름을 변경 가능
df = df.rename(columns={0: 'Count'}).rename_axis('idx')
df

,Gu,Dong,Category,Current_License_info,Count
idx,,,,,
0,강남구,논현동,운송,2018,2
1,강남구,논현동,운송,2019,1
2,강남구,논현동,운송,2021,2
3,강남구,논현동,운송,2022,1
4,강남구,논현동,운송,2024,1
...,...,...,...,...,...
293,중랑구,면목동,운송,2024,1
294,중랑구,묵동,운송,2022,1
295,중랑구,묵동,운송,2023,2


In [265]:
print(df['Count'].isna().sum())  # 결측값 개수 확인
print(df['Count'].isin([np.inf, -np.inf]).sum())  # 무한대 값 개수 확인


0
0


In [266]:
df.to_csv('운송업_Year_Gu_Dong_Counts_지우.csv')

In [267]:
# 인덱스 오류가 있을때, reset_index() 함수를 사용하여 그룹화된 결과를 다시 인덱스를 기준으로 정렬
# reset_index(name='Counts')['Counts']
# : 위 부분은 그룹화된 결과를 다시 인덱스를 기준으로 정렬하고,
# : Counts 열만 선택하여 Series로 변환

In [268]:
# 구/ 동/ 카테고리를 그룹화한 수를 포함한 데이터

In [269]:
# 데이터 확인
transportation.head()

,Gu,Dong,Category,Current_License_info
0,강동구,길동,운송,2024
1,강동구,명일동,운송,2024
2,강동구,성내동,운송,2024
3,강동구,강일동,운송,2024
4,강동구,성내동,운송,2023


In [270]:
# 필요한 행만 추출
df1 = transportation[['Gu','Dong','Category']]

df1

,Gu,Dong,Category
0,강동구,길동,운송
1,강동구,명일동,운송
2,강동구,성내동,운송
3,강동구,강일동,운송
4,강동구,성내동,운송
...,...,...,...
431,용산구,한강로2가,운송
432,용산구,보광동,운송
433,중구,신당동,운송
434,종로구,충신동,운송


In [271]:
# 구, 동, 카테고리 별 수
# df1 = 구, 동, 카테고리 별 수를 포함한 데이터
df1 = df1.groupby(['Gu','Dong','Category']).size()

# 열의 이름을 변경하기 위해 데이터프레임의 형식으로 변경
df1 = pd.DataFrame(df1)

df1 = df1.reset_index()

# 열 이름 변경
df1 = df1.rename(columns = {0:'Count'}).rename_axis('idx')
df1

,Gu,Dong,Category,Count
idx,,,,
0,강남구,논현동,운송,7
1,강남구,대치동,운송,1
2,강남구,도곡동,운송,2
3,강남구,수서동,운송,1
4,강남구,역삼동,운송,4
...,...,...,...,...
144,중랑구,망우동,운송,4
145,중랑구,면목동,운송,6
146,중랑구,묵동,운송,3


In [272]:
print(df1['Count'].isna().sum())  # 결측값 개수 확인
print(df1['Count'].isin([np.inf, -np.inf]).sum())  # 무한대 값 개수 확인

0
0


In [273]:
df1.to_csv('운송업_Gu_Dong_Counts_지우.csv')